# Chatbot RAG avec PostgreSQL, pgvector et Groq

Ce notebook implémente un système RAG (Retrieval Augmented Generation) utilisant :
- **PostgreSQL** avec **pgvector** pour le stockage vectoriel
- **Sentence-Transformers** pour les embeddings gratuits
- **Groq API** pour la génération de réponses (gratuit)
- **psycopg3** pour la connexion à PostgreSQL

##  Installation et Import des bibliothèques

In [ ]:
import sys
import subprocess

packages = [
    'psycopg[binary]',
    'pgvector',
    'sentence-transformers',
    'groq',
    'python-dotenv',
    'numpy',
    'pandas'
]

print("Installation des packages en cours...")
for package in packages:
    try:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', '-q', package])
        print(f"✓ {package} installé")
    except Exception as e:
        print(f"✗ Erreur pour {package}: {e}")

print("\n✓ Installation terminée !")

Installation des packages en cours...
✓ psycopg[binary] installé
✓ pgvector installé
✓ sentence-transformers installé
✓ groq installé
✓ python-dotenv installé
✓ numpy installé
✓ pandas installé

✓ Installation terminée !


## Imports et Configuration

In [ ]:
import psycopg
from psycopg import sql
import numpy as np
import pandas as pd
from sentence_transformers import SentenceTransformer
from groq import Groq
import os
from dotenv import load_dotenv
import requests
from bs4 import BeautifulSoup

# Charger les variables d'environnement
load_dotenv('../src/.env')

print("✓ Bibliothèques importées avec succès")

c:\Users\fadou\OneDrive\Bureau\Rag chatbot\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


✓ Bibliothèques importées avec succès


##  Connexion à PostgreSQL

In [ ]:
try:
    conn.close()
    print("✓ Ancienne connexion fermée")
except:
    print("Connexion déjà fermée")


DB_CONFIG = {
    'host': os.getenv('DB_HOST', 'localhost'),
    'port': os.getenv('DB_PORT', '5432'),
    'dbname': os.getenv('DB_NAME', 'rag_chatbot'),
    'user': os.getenv('DB_USER', 'postgres'),
    'password': os.getenv('DB_PASSWORD', ''),
    'autocommit': True 
}

conn = psycopg.connect(**DB_CONFIG)
print("✓ Nouvelle connexion créée (autocommit=True)")
print("✓ La transaction est maintenant propre")
print("\n" + "="*80)
print(" ✓ Vous pouvez maintenant exécuter les cellules suivantes")
print("="*80)

⚠️ Connexion déjà fermée
✓ Nouvelle connexion créée (autocommit=True)
✓ La transaction est maintenant propre

✅ Vous pouvez maintenant exécuter les cellules suivantes


##  Activer l'extension pgvector

In [ ]:
with conn.cursor() as cur:
    try:
        cur.execute("CREATE EXTENSION IF NOT EXISTS vector;")
        conn.commit()
        print("✓ Extension pgvector activée")
    except Exception as e:
        print(f"✗ Erreur: {e}")
        conn.rollback()

✓ Extension pgvector activée


##  Créer la table pour les dialogues

In [ ]:
print("\n🔄 Création de la table 'dialogues'...")

with conn.cursor() as cur:
    # Supprimer l'ancienne table si elle existe
    cur.execute("DROP TABLE IF EXISTS dialogues CASCADE;")
    
    # Créer la nouvelle table
    cur.execute("""
        CREATE TABLE dialogues (
            id SERIAL PRIMARY KEY,
            dialogue_id VARCHAR(50),
            contenu TEXT NOT NULL,
            embedding vector(384)
        );
    """)
    
    # Créer un index pour accélérer les recherches vectorielles
    cur.execute("""
        CREATE INDEX ON dialogues USING ivfflat (embedding vector_cosine_ops)
        WITH (lists = 100);
    """)

print("✓ Table 'dialogues' créée avec succès")
print("  - Colonnes: id, dialogue_id, contenu, embedding (vector 384)")


🔄 Création de la table 'dialogues'...
✓ Table 'dialogues' créée avec succès
  - Colonnes: id, dialogue_id, contenu, embedding (vector 384)


## Charger le modèle d'embedding multilingue

In [7]:
# Charger le modèle d'embedding multilingue
print("\n🔄 Chargement du modèle d'embedding...")
model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')

print("✓ Modèle 'paraphrase-multilingual-MiniLM-L12-v2' chargé (384 dimensions)")
print("  Ce modèle est optimisé pour le français et 115 autres langues")


🔄 Chargement du modèle d'embedding...
✓ Modèle 'paraphrase-multilingual-MiniLM-L12-v2' chargé (384 dimensions)
  Ce modèle est optimisé pour le français et 115 autres langues


## Lire les dialogues depuis data/

In [ ]:
from pathlib import Path
import re

print("\n📂 LECTURE DES DIALOGUES\n")

data_dir = Path("../data")
tous_dialogues = []
dialogue_ids = []

for filepath in sorted(data_dir.glob("*.txt")):
    print(f"📄 Lecture de {filepath.name}...")
    
    with open(filepath, 'r', encoding='latin-1') as f:
        contenu = f.read()

    # Séparer les dialogues par des lignes vides multiples
    dialogues_bruts = re.split(r'\n\n+', contenu)
    
    count = 0
    for i, dialogue in enumerate(dialogues_bruts):
        dialogue = dialogue.strip()
        
        # Garder seulement les dialogues avec au moins 100 caractères
        if dialogue and len(dialogue) > 100:
            tous_dialogues.append(dialogue)
            dialogue_id = f"{filepath.stem}_part{i+1}"
            dialogue_ids.append(dialogue_id)
            count += 1
    
    print(f"   ✓ {count} dialogues extraits")

print(f"\n{'='*80}")
print(f" TOTAL: {len(tous_dialogues)} dialogues chargés")
print(f"{'='*80}")

# Aperçu du premier dialogue
if tous_dialogues:
    print("\n Aperçu du 1er dialogue:")
    print("-" * 80)
    print(tous_dialogues[0][:300] + "...")
    print("-" * 80)

# Statistiques
longueurs = [len(d) for d in tous_dialogues]
print(f"\n Statistiques:")
print(f"   - Nombre de dialogues: {len(tous_dialogues)}")
print(f"   - Longueur moyenne: {sum(longueurs) / len(longueurs):.0f} caractères")
print(f"   - Dialogue le plus court: {min(longueurs)} caractères")
print(f"   - Dialogue le plus long: {max(longueurs)} caractères")



📂 LECTURE DES DIALOGUES

📄 Lecture de 017_00000012.txt...
   ✓ 1 dialogues extraits
📄 Lecture de 018_00000013.txt...
   ✓ 1 dialogues extraits
📄 Lecture de 019_00000014.txt...
   ✓ 1 dialogues extraits
📄 Lecture de 020_00000015.txt...
   ✓ 1 dialogues extraits
📄 Lecture de 022_00000017.txt...
   ✓ 1 dialogues extraits
📄 Lecture de 023_00000018.txt...
   ✓ 1 dialogues extraits
📄 Lecture de 024_00000019.txt...
   ✓ 1 dialogues extraits
📄 Lecture de 027_0000001c.txt...
   ✓ 1 dialogues extraits
📄 Lecture de 028_0000001d.txt...
   ✓ 1 dialogues extraits
📄 Lecture de 029_0000001e.txt...
   ✓ 1 dialogues extraits
📄 Lecture de 030_0000001f.txt...
   ✓ 1 dialogues extraits
📄 Lecture de 037_00000026.txt...
   ✓ 1 dialogues extraits
📄 Lecture de 038_00000027.txt...
   ✓ 1 dialogues extraits
📄 Lecture de 045_0000002e.txt...
   ✓ 1 dialogues extraits
📄 Lecture de 047_00000030.txt...
   ✓ 1 dialogues extraits
📄 Lecture de 048_00000031.txt...
   ✓ 1 dialogues extraits
📄 Lecture de 052_00000035.txt.

## Générer les embeddings

In [13]:
# Générer les embeddings pour tous les dialogues
print("\n" + "="*80)
print("🔄 GÉNÉRATION DES EMBEDDINGS")
print("="*80)

print(f"\nModèle: paraphrase-multilingual-MiniLM-L12-v2")
print(f"Dialogues à encoder: {len(tous_dialogues)}\n")

embeddings = model.encode(
    tous_dialogues,
    batch_size=16,
    show_progress_bar=True,
    normalize_embeddings=True  # Normalisation pour améliorer la similarité cosinus
)

print(f"\n✓ {len(embeddings)} embeddings générés")
print(f"✓ Shape: {embeddings.shape}")
print(f"✓ Type: {embeddings.dtype}")


🔄 GÉNÉRATION DES EMBEDDINGS

Modèle: paraphrase-multilingual-MiniLM-L12-v2
Dialogues à encoder: 40



Batches: 100%|██████████| 3/3 [00:01<00:00,  1.53it/s]


✓ 40 embeddings générés
✓ Shape: (40, 384)
✓ Type: float32


##  Insérer dans PostgreSQL

In [14]:
# Insérer les dialogues dans PostgreSQL
from pgvector.psycopg import register_vector

print("\n" + "="*80)
print("💾 INSERTION DANS POSTGRESQL")
print("="*80)

# Enregistrer le type vector
register_vector(conn)
print("\n✓ Type 'vector' enregistré\n")

# Insérer les dialogues
print("Insertion en cours...")
with conn.cursor() as cur:
    for i, (dialogue_id, dialogue, emb) in enumerate(zip(dialogue_ids, tous_dialogues, embeddings), 1):
        # Conversion en float Python natifs
        emb_list = [float(x) for x in emb]
        
        cur.execute(
            "INSERT INTO dialogues (dialogue_id, contenu, embedding) VALUES (%s, %s, %s)",
            (dialogue_id, dialogue, emb_list)
        )
        
        if i % 10 == 0:
            print(f"   ✓ {i}/{len(tous_dialogues)} dialogues insérés...")

print(f"\n✅ {len(tous_dialogues)} dialogues insérés avec succès!")

# Vérification finale
with conn.cursor() as cur:
    cur.execute("SELECT COUNT(*) FROM dialogues;")
    count = cur.fetchone()[0]
    print(f"✓ Vérification: {count} dialogues dans la base")


💾 INSERTION DANS POSTGRESQL

✓ Type 'vector' enregistré

Insertion en cours...
   ✓ 10/40 dialogues insérés...
   ✓ 20/40 dialogues insérés...
   ✓ 30/40 dialogues insérés...
   ✓ 40/40 dialogues insérés...

✅ 40 dialogues insérés avec succès!
✓ Vérification: 40 dialogues dans la base


## Définir la fonction de recherche

In [15]:
# Fonction de recherche de dialogues similaires
def search_similar_dialogues(query, top_k=3):
    """
    Recherche les dialogues les plus similaires à la requête
    """
    # Générer l'embedding de la requête avec normalisation
    query_embedding = model.encode(query, normalize_embeddings=True)
    
    # Convertir en float Python natifs
    embedding_list = [float(x) for x in query_embedding]
    
    # Rechercher dans la base avec similarité cosinus
    with conn.cursor() as cur:
        cur.execute("""
            SELECT id, dialogue_id, contenu, 1 - (embedding <=> %s::vector) AS similarity
            FROM dialogues
            ORDER BY similarity DESC
            LIMIT %s;
        """, (embedding_list, top_k))
        
        results = cur.fetchall()
    
    return results

print("✓ Fonction search_similar_dialogues définie")

✓ Fonction search_similar_dialogues définie


 ## Tester la recherche

In [19]:
# Test de recherche
print("\n" + "="*80)
print("🧪 TEST DE RECHERCHE")
print("="*80)

test_queries = [
    "Comment saluer un client au début de l'appel ?",
   
]

for query in test_queries:
    print(f"\n❓ Query: '{query}'")
    results = search_similar_dialogues(query, top_k=3)
    
    print()
    for i, (doc_id, dialogue_id, text, similarity) in enumerate(results, 1):
        # Indicateur de qualité
        if similarity > 0.7:
            indicator = "🟢 EXCELLENT"
        elif similarity > 0.5:
            indicator = "🟡 BON"
        elif similarity > 0.3:
            indicator = "🟠 MOYEN"
        else:
            indicator = "🔴 FAIBLE"
        
        print(f"  {i}. {indicator} [Score: {similarity:.3f}]")
        print(f"     ID: {dialogue_id}")
        print(f"     Extrait: {text[:100]}...\n")


🧪 TEST DE RECHERCHE

❓ Query: 'Comment saluer un client au début de l'appel ?'

  1. 🟡 BON [Score: 0.659]
     ID: 022_00000017_part1
     Extrait: <01> hotesse
     h: U B S bonjour
<02> client
     c: oui bonjour est-ce que je pourrais avoir mons...

  2. 🟡 BON [Score: 0.585]
     ID: 023_00000018_part1
     Extrait: <01> hotesse
     h: U B S bonjour
<02> client
     c: oui bonjour excusez moi de vous déranger est ...

  3. 🟡 BON [Score: 0.582]
     ID: 087_00000058_part1
     Extrait: <01> hotesse
     h: U B S bonjour
<02> client
     c: oui bonjour madame j'aurais voulu parler à Pr...



## Configuration de Groq API

In [20]:
# Configuration de Groq
groq_api_key = os.getenv('GROQ_API_KEY')

if not groq_api_key:
    print("⚠ GROQ_API_KEY non trouvée dans le fichier .env")
    print("  Obtenez votre clé gratuite sur: https://console.groq.com")
else:
    client = Groq(api_key=groq_api_key)
    print("✓ Client Groq initialisé")
    print("  Modèle: llama-3.3-70b-versatile")

✓ Client Groq initialisé
  Modèle: llama-3.3-70b-versatile


##   Fonction RAG complète

In [22]:
# Fonction RAG complète pour dialogues
def rag_chatbot_dialogues(user_question, top_k=3):
    """
    Chatbot RAG utilisant Groq pour répondre basé sur des dialogues
    """
    # 1. Rechercher les dialogues pertinents
    print(f"🔍 Recherche de dialogues pertinents...")
    similar_dialogues = search_similar_dialogues(user_question, top_k=top_k)
    
    if not similar_dialogues:
        print("⚠️ Aucun dialogue trouvé!")
        return "Désolé, je n'ai pas trouvé de dialogue pertinent dans ma base."
    
    # 2. Créer le contexte
    context = "\n\n".join([dialogue[2] for dialogue in similar_dialogues])
    print(f"✓ {len(similar_dialogues)} dialogues trouvés\n")
    
    # Afficher les dialogues trouvés
    print("Dialogues pertinents:")
    for i, (doc_id, dialogue_id, text, similarity) in enumerate(similar_dialogues, 1):
        print(f"  {i}. [Similarité: {similarity:.3f}] {dialogue_id}")
        print(f"     {text[:80]}...")
    print()
    
    # 3. Créer le prompt avec contexte
    prompt = f"""Tu es un assistant intelligent qui répond aux questions en te basant sur des dialogues de conversations téléphoniques.

Contexte (dialogues de conversations):
{context}

Question: {user_question}

Réponds de manière claire et concise en français, en t'appuyant sur les informations contenues dans les dialogues."""

    # 4. Appeler Groq pour générer la réponse
    print("🤖 Génération de la réponse avec Groq...")
    try:
        chat_completion = client.chat.completions.create(
            messages=[{"role": "user", "content": prompt}],
            model="llama-3.3-70b-versatile",
            temperature=0.7,
            max_tokens=500,
        )
        
        response = chat_completion.choices[0].message.content
        return response
    
    except Exception as e:
        return f"Erreur lors de la génération: {e}"

print("✓ Fonction rag_chatbot_dialogues définie")

✓ Fonction rag_chatbot_dialogues définie


## Test final du chatbot

In [25]:
# Test du chatbot RAG sur dialogues
question = "Comment saluer un client au début de l'appel ?"

print("="*70)
print(f"Question: {question}")
print("="*70)

response = rag_chatbot_dialogues(question)

print("\n" + "="*70)
print("Réponse:")
print("="*70)
print(response)
print("="*70)

Question: Comment saluer un client au début de l'appel ?
🔍 Recherche de dialogues pertinents...
✓ 3 dialogues trouvés

Dialogues pertinents:
  1. [Similarité: 0.659] 022_00000017_part1
     <01> hotesse
     h: U B S bonjour
<02> client
     c: oui bonjour est-ce que je...
  2. [Similarité: 0.585] 023_00000018_part1
     <01> hotesse
     h: U B S bonjour
<02> client
     c: oui bonjour excusez moi d...
  3. [Similarité: 0.582] 087_00000058_part1
     <01> hotesse
     h: U B S bonjour
<02> client
     c: oui bonjour madame j'aura...

🤖 Génération de la réponse avec Groq...

Réponse:
Pour saluer un client au début de l'appel, il suffit de dire "bonjour" comme indiqué dans les dialogues, par exemple : "U B S bonjour".


##  Fermeture de la connexion

In [ ]:
# Fermer la connexion
conn.close()
print("✓ Connexion PostgreSQL fermée")